# Trabajo Final de Data Science 2024-2025:

El trabajo consiste en analizar un dataset libremente elegido por cada grupo, que constará de 2 alumnos.

El objetivo es que cada grupo sea capaz de llevar a cabo un análisis riguroso de los datos, así como de exponer y visualizar las conclusiones alcanzadas de forma convincente.

El trabajo puede llevarse a cabo tanto en R como en Python. Se entregará un ZIP file script con el código, datos necesarios y un informe del trabajo.

Se valorará los siguientes apartados:

1. Correcta definición del estudio que se quiere llevar a cabo, estableciendo claramente las preguntas que se pretender analizar en el estudio.

2. Uso adecuado de las distintas técnicas analíticas explicada durante el curso, incluyendo estadística inferencial y contraste de hipótesis, PCA, modelos de regresión lineal, incluyendo regularización, modelos de regresión logística, selección de variables, métodos de clustering.

3. Uso adecuado de técnicas de procesamiento de datos y visualización explicadas durante el curso.

4. Estilo adecuado y profesional en el informe entregado. Puede ser tanto en inglés como en castellano.

5. Presentación oral de los resultados del trabajo.  

Se adjunta un ejemplo de trabajo realizado el curso anterior. No es un trabajo perfecto. Tiene algunos fallos, particularmente se echan en falta un mejor uso de contraste de hipótesis para sustanciar los resultados e hipótesis generadas. Sin embargo, sirve como ejemplo para ver lo que se pide.

Fecha de Entrega: 06/11/2024

Fecha de Presentaciones: 12/11/2024, 15:00

## Datos:
(Jaime)
(explicación del dataset: columnas, tipos de datos, tamaño, fuente de los datos(link)...)

## Metodología:

- Análisis del problema ¿Qué queremos?
- Visualización de los datos en crudo.
- Procesado de los datos.
- Representación de los datos y sus relaciones. (EDA)
- Contraste de hipotesis (Luis).
- Todo el tema de estadistica y machine learning.


# Anotaciones del trabajo:

tenemos 1320 filas y 20 columnas

* Columnas a borrar en el dataset:
    - User ID
    - Charging Start Time
    - Charging End Time



Preguntas:

- ¿Cuáles son las principales variables que influyen en el tiempo total de carga?


Gráficas sencillas:

- Picos de carga agrupados en momento de día y día: 7*4= 28 momentos
- Matriz de relaciones que influye más en la duración de la carga o en el coste. (solo datos numericos).

Preguntas sacadas de internet (Jaime):

* Análisis de comportamiento de carga:

¿Cuál es la variación del tiempo de carga en función de diferentes horas del día o días de la semana?
¿Existe una relación entre el tiempo de carga y la tarifa de energía en distintas estaciones o franjas horarias?
¿Se observan patrones estacionales en el uso de estaciones de carga (por ejemplo, diferencias entre invierno y verano)?

* Segmentación de usuarios y patrones de uso:

¿Existen diferentes perfiles de usuarios (según tiempo medio de carga, frecuencia de uso, etc.) y cómo varían sus comportamientos?
¿Qué diferencias existen en los patrones de carga entre usuarios particulares y comerciales, si el dataset contiene esta información?
¿Qué factores parecen influir más en la elección de una estación de carga específica?

* Predicción y optimización:

¿Qué variables tienen el mayor impacto en la duración del tiempo de carga y en la frecuencia de uso de las estaciones?
¿Es posible predecir las horas pico de uso en función de los datos históricos?
¿Qué características del punto de carga (como potencia, ubicación) predicen un mayor número de usuarios?

* Impacto en la infraestructura de carga:

¿Cuáles son los puntos de carga con mayor uso, y qué características tienen en común?
¿Cómo varía la demanda de carga según la proximidad a áreas urbanas o rurales?
¿Existen patrones de uso que sugieren la necesidad de ampliar la infraestructura en ciertas áreas?

* Sostenibilidad y Eficiencia Energética:

¿Cuánto tiempo, en promedio, permanecen los vehículos conectados después de estar completamente cargados?
¿Cuál es la tasa de eficiencia en el uso de las estaciones de carga? ¿Se producen períodos significativos de inactividad?
¿Cómo afecta el tiempo de conexión a la eficiencia de la infraestructura de carga en general?

* Tendencias de adopción de EV:

¿Se observa un crecimiento en la cantidad de sesiones de carga a lo largo del tiempo? ¿Es consistente en todas las estaciones?
¿Qué patrones emergentes indican un aumento en la adopción de vehículos eléctricos en ciertas áreas o tipos de usuarios?

Posibles Preguntas de Análisis (Luis)
* Patrones de Consumo y Eficiencia:
-¿Cuál es la relación entre el Estado de Carga al inicio y final y la Energía Consumida?
-¿Cómo varía el Costo de Carga en función del Ubicación de la Estación o el Tipo de Cargador?

* Factores que Afectan la Duración y Costo de la Carga:
¿Cómo influyen el Modelo de Vehículo, la Capacidad de Batería y la Edad del Vehículo en el Costo de Carga y el Tiempo de Carga?
¿Qué impacto tiene la Temperatura en la Duración de Carga y Eficiencia?

* Análisis de Uso del Cargador por Tipo de Usuario:
¿Qué diferencias existen entre los patrones de carga de los usuarios clasificados como Commuters frente a los Long-Distance Travelers?
¿Existe una tendencia en el Horario de Carga según el Tipo de Usuario?

* Análisis de Localización y Estaciones de Carga:
¿Cuáles son los lugares con mayor consumo de energía, y cómo varía el Tiempo de Carga entre distintas Ubicaciones?
¿Se identifican patrones de alta demanda según el Día de la Semana o la Hora del Día?


In [1]:
import numpy as np
import pandas as pd
import pylab as pl
import matplotlib.pyplot as plt

raw_data = pd.read_csv('DatasetDS/ev_charging_patterns.csv')
print(raw_data.head(5))
print(raw_data.describe())
print(f"Columnas numericas = {raw_data.describe().columns}")

for column in ['Battery Capacity (kWh)', 'Energy Consumed (kWh)',
               'Charging Duration (hours)', 'Charging Rate (kW)',
               'Charging Cost (USD)', 'State of Charge (Start %)',
               'State of Charge (End %)', 'Distance Driven (since last charge) (km)',
               'Temperature (°C)', 'Vehicle Age (years)']:
    raw_data[column].fillna(raw_data[column].median(), inplace=True)

# Verificación para comprobar que no hay NaN values
print(raw_data.isna().sum())

# Redondear valores de Battery Capacity (kWh) y Vehicle Age (years) a 0 decimales
raw_data['Battery Capacity (kWh)'] = raw_data['Battery Capacity (kWh)'].round(0)
raw_data['Vehicle Age (years)'] = raw_data['Vehicle Age (years)'].round(0)

# Verifica los cambios en las columnas
print(raw_data[['Battery Capacity (kWh)', 'Vehicle Age (years)']].head())

# Crear la columna 'Percentage of Charge' y eliminar columnas
raw_data['Percentage of Charge'] = raw_data['State of Charge (End %)'] - raw_data['State of Charge (Start %)']
data_cleaned = raw_data.drop(columns=['User ID', 'Charging Station ID', 'Charging Start Time', 'Charging End Time', 'State of Charge (Start %)', 'State of Charge (End %)'])

# Mostrar el DataFrame resultante
data_cleaned

  User ID Vehicle Model  Battery Capacity (kWh) Charging Station ID  \
0  User_1        BMW i3              108.463007         Station_391   
1  User_2  Hyundai Kona              100.000000         Station_428   
2  User_3    Chevy Bolt               75.000000         Station_181   
3  User_4  Hyundai Kona               50.000000         Station_327   
4  User_5  Hyundai Kona               50.000000         Station_108   

  Charging Station Location  Charging Start Time    Charging End Time  \
0                   Houston  2024-01-01 00:00:00  2024-01-01 00:39:00   
1             San Francisco  2024-01-01 01:00:00  2024-01-01 03:01:00   
2             San Francisco  2024-01-01 02:00:00  2024-01-01 04:48:00   
3                   Houston  2024-01-01 03:00:00  2024-01-01 06:42:00   
4               Los Angeles  2024-01-01 04:00:00  2024-01-01 05:46:00   

   Energy Consumed (kWh)  Charging Duration (hours)  Charging Rate (kW)  \
0              60.712346                   0.591363        

,Vehicle Model,Battery Capacity (kWh),Charging Station Location,Energy Consumed (kWh),Charging Duration (hours),Charging Rate (kW),Charging Cost (USD),Time of Day,Day of Week,Distance Driven (since last charge) (km),Temperature (°C),Vehicle Age (years),Charger Type,User Type,Percentage of Charge
0,BMW i3,108.0,Houston,60.712346,0.591363,36.389181,13.087717,Evening,Tuesday,293.602111,27.947953,2.0,DC Fast Charger,Commuter,56.748386
1,Hyundai Kona,100.0,San Francisco,12.339275,3.133652,30.677735,21.128448,Morning,Monday,112.112804,14.311026,3.0,Level 1,Casual Driver,74.548566
2,Chevy Bolt,75.0,San Francisco,19.128876,2.452653,27.513593,35.667270,Morning,Thursday,71.799253,21.002002,2.0,Level 2,Commuter,63.063011
3,Hyundai Kona,50.0,Houston,79.457824,1.266431,32.882870,13.036239,Evening,Saturday,199.577785,38.316313,1.0,Level 1,Long-Distance Traveler,16.504325
4,Hyundai Kona,50.0,Los Angeles,19.629104,2.019765,10.215712,10.161471,Morning,Saturday,203.661847,-7.834199,1.0,Level 1,Long-Distance Traveler,9.484836
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1315,Nissan Leaf,100.0,New York,42.011654,1.426444,5.895475,22.081164,Evening,Sunday,239.601075,1.919655,7.0,DC Fast Charger,Commuter,44.711850
1316,BMW i3,100.0,New York,68.185853,3.238212,18.388012,5.067806,Evening,Tuesday,164.376022,34.029775,4.0,Level 2,Casual Driver,61.640087
1317,Nissan Leaf,100.0,New York,18.895102,3.267122,45.482066,37.255002,Evening,Tuesday,226.519258,20.358761,5.0,DC Fast Charger,Commuter,6.775798
1318,Chevy Bolt,85.0,San Francisco,13.756252,2.754527,38.148183,39.046146,Afternoon,Sunday,291.494076,24.134598,5.0,Level 2,Commuter,-10.261424


Vamos a realizar un ejemplo de test ANOVA para comparar el costo promedio de carga entre distintas ubicaciones de estación. Esto nos ayudará a determinar si hay diferencias significativas en el costo de carga dependiendo del lugar.

In [2]:
from scipy.stats import f_oneway

# Filtrar datos necesarios para el ANOVA
# Agrupar los datos de "Charging Cost (USD)" por "Charging Station Location"
anova_data = data_cleaned[['Charging Station Location', 'Charging Cost (USD)']].dropna()

# Extraer los costos por cada ubicación única
station_groups = [anova_data['Charging Cost (USD)'][anova_data['Charging Station Location'] == location] 
                  for location in anova_data['Charging Station Location'].unique()]

# Realizar el test ANOVA
anova_result = f_oneway(*station_groups)
anova_result


F_onewayResult(statistic=1.0067592060834571, pvalue=0.4027757556301411)

Un valor bajo de F, como 1.007, indica que la variabilidad entre los grupos es pequeña en comparación con la variabilidad dentro de cada grupo.

Esto significa que las diferencias en el costo de carga entre las distintas ubicaciones son pequeñas y no mayores a la variabilidad dentro de cada ubicación.

Vamos a realizar una prueba t para comparar el tiempo de carga promedio entre dos tipos de cargadores

In [7]:
data_cleaned['Charger Type'].unique()

array(['DC Fast Charger', 'Level 1', 'Level 2'], dtype=object)

In [8]:
from scipy.stats import f_oneway

# Agrupar los datos de tiempos de carga por tipo de cargador
dc_fast = data_cleaned[data_cleaned['Charger Type'] == 'DC Fast Charger']['Charging Duration (hours)'].dropna()
level_1 = data_cleaned[data_cleaned['Charger Type'] == 'Level 1']['Charging Duration (hours)'].dropna()
level_2 = data_cleaned[data_cleaned['Charger Type'] == 'Level 2']['Charging Duration (hours)'].dropna()

# Realizar el test ANOVA
anova_result = f_oneway(dc_fast, level_1, level_2)
print("Estadístico F:", anova_result.statistic)
print("Valor p:", anova_result.pvalue)

Estadístico F: 0.21924114697699768
Valor p: 0.8031573287790372


Un valor de F tan bajo indica que la variabilidad en el tiempo de carga entre los distintos tipos de cargador es muy pequeña en comparación con la variabilidad dentro de cada tipo.

Un valor p de 0.803, que es mucho mayor que el umbral de 0.05, significa que no existe evidencia estadísticamente significativa para rechazar la hipótesis nula. En este caso, la hipótesis nula supone que no hay diferencias significativas en el tiempo de carga promedio entre los tres tipos de cargadores.

Contraste de hipótesis basado en la relación entre la temperatura y el consumo de energía utilizando un análisis de correlación. Esto nos permitirá ver si existe una correlación significativa entre la Temperatura y la Energía Consumida (kWh)

In [5]:
from scipy.stats import pearsonr

correlation_data = data_cleaned[['Temperature (°C)', 'Energy Consumed (kWh)']].dropna()

correlation_result, p_value = pearsonr(correlation_data['Temperature (°C)'], correlation_data['Energy Consumed (kWh)'])
print("Correlación de Pearson:", correlation_result)
print("Valor p:", p_value)


Correlación de Pearson: -0.01799274721878756
Valor p: 0.5136643975700691


El valor de correlación de Pearson es muy cercano a 0, lo que indica que no hay una correlación lineal significativa entre la temperatura y el consumo de energía. Un valor negativo muy bajo sugiere una relación casi nula.

El valor p es bastante alto (mayor que el nivel de significancia común de 0.05), lo que significa que no podemos rechazar la hipótesis nula. En este caso, la hipótesis nula establece que no existe una correlación significativa entre la temperatura y el consumo de energía.